This is my final project.
The goal is to take a table of data on a lot of contrys and use it to predict a countrys co2 output at a given time. 


In [313]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
from sklearn import linear_model
DF = pd.read_csv("energy.csv")
DF.head(7)


,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000
5,5,World,renewables_n_other,1980,20.702344,20.775178,27770.910281,4.298127e+06,68.145921,10.547,0.000000
6,6,Afghanistan,all_energy_types,1980,0.026583,0.072561,NaN,1.335650e+04,1.990283,0.000,NaN


So im going to creat a model that takes the Energg type, year, gdp population and trys to predict the Energy consumption. 

In [314]:
tolist = DF.shape
DFlist = list(tolist)
print(DFlist)
rowsofdata = DFlist[0]
print(type(rowsofdata))

[55440, 11]
<class 'int'>


In [315]:
#cleaning data as we only need some data

CDF = DF[["Energy_type","Year",	"Energy_consumption",	"GDP",	"Population"]]

CDF.head(7)
print(CDF["Energy_type"])
#removing all non genral energy types
CDF = CDF[(CDF['Energy_type'] == 'all_energy_types')]
#removing all rows with "NaN"
CDF = CDF.dropna()
CDF.head(rowsofdata)


0                 all_energy_types
1                             coal
2                      natural_gas
3        petroleum_n_other_liquids
4                          nuclear
                   ...            
55435                         coal
55436                  natural_gas
55437    petroleum_n_other_liquids
55438                      nuclear
55439           renewables_n_other
Name: Energy_type, Length: 55440, dtype: object


,Energy_type,Year,Energy_consumption,GDP,Population
0,all_energy_types,1980,292.899790,27770.910281,4.298127e+06
48,all_energy_types,1980,1.713088,453.819000,2.795193e+04
66,all_energy_types,1980,2.985436,373.576200,1.472355e+04
72,all_energy_types,1980,1.097383,212.487100,7.550408e+03
108,all_energy_types,1980,2.122077,267.996100,9.860275e+03
...,...,...,...,...,...
55398,all_energy_types,2019,2.447475,190.147200,2.851585e+04
55404,all_energy_types,2019,3.509086,915.504000,9.642590e+04
55422,all_energy_types,2019,0.159020,53.083800,2.916220e+04
55428,all_energy_types,2019,0.207097,247.181900,1.787385e+04


So now we have taken ought all the data that is about spisific energy types, and all the data that has a null value
Next im removing the column Energyt type

In [316]:
fdf = CDF[["Year",	"Energy_consumption",	"GDP",	"Population"]]

In [317]:
msk = np.random.rand(len(fdf)) < 0.8
train = fdf[msk]
test = fdf[~msk]

regr = linear_model.LinearRegression()
# we put all indipents into the x, 
X = np.asanyarray(train[["Year", "GDP",	"Population"]])
y = np.asanyarray(train[["Energy_consumption"]])
regr.fit (X, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[-4.18857964e-02  3.52913291e-03  2.79775952e-05]]


Scoring it

In [318]:
y_hat= regr.predict(test[["Year",	"GDP",	"Population"]])
x_seprates = np.asanyarray(test[["Year",	"GDP",	"Population"]])
y = np.asanyarray(test[["Energy_consumption"]])
print("Mean Squared Error (MSE) : %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score for together is : %.2f' % regr.score(x_seprate, y))
#this is a variable for comparing them latter
combo = regr.score(x_seprate, y)

Mean Squared Error (MSE) : 24.60
Variance score for together is : 0.96


C:\Users\kaica\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


so now we have to know how to use the model. 
